# 1. Connect to JIRA

Sets the matplotlib outputs to be rendered inline and loads library code into the kernel. Update the `JIRA_HOST` and `JIRA_USERNAME` before running the cell. You may also need to trust this notebook; read through `code.py` before you do that. 

In [1]:
!pip install -r requirements.txt

     |████████████████████████████████| 61 kB 355 kB/s eta 0:00:01
     |████████████████████████████████| 54 kB 903 kB/s eta 0:00:01
     |████████████████████████████████| 119 kB 1.3 MB/s eta 0:00:01
     |████████████████████████████████| 45 kB 281 kB/s eta 0:00:01
     |████████████████████████████████| 54 kB 292 kB/s eta 0:00:01


In [20]:
%matplotlib inline
%run utils.py

from getpass import getpass

import pandas as pd
from pandas import DataFrame

JIRA_HOST = 'https://jira.wiley.com/'
JIRA_USERNAME = 'im_cpp_bot'

jira_client = connect(JIRA_HOST, JIRA_USERNAME, getpass(prompt='Enter JIRA password: '))

print('Connected to {client_info}'.format(client_info=jira_client.client_info()))

Enter JIRA password: ········
Connected to https://jira.wiley.com


In [ ]:
!pip install dateparser
!pip install pytz

In [122]:
import dateparser as dp
import pytz

timezone_nw = pytz.timezone('America/New_York')

def read_data_sets(jql, max_results=10000, status_alias=None):
    if status_alias is None:
        status_alias = {}

    issues = jira_client.search_issues(jql, maxResults=max_results, expand='changelog')
    rows = []
    for issue in issues:
        # Add the issue creation as the fist event transition           
        try:
            epic = issue.raw['fields']['customfield_16525'] or 'empty'
        except Exception as e:
            epic = 'empty'
    
        # Issue creation (None -> Backlog)
        rows.append({'key': issue.key,
                     'ts': dp.parse(issue.fields.created).astimezone(timezone_nw),
                     'from': None,
                     'to': status_alias.get('Backlog') or 'Backlog'})
        
        # Other transitions (Eg: Backlog to Development)
        for history in issue.changelog.histories:
            for item in history.items:
                if item.field == 'status':
                    rows.append({'key': issue.key,
                                 'epic': epic or 'empty',
                                 'ts': dp.parse(history.created).astimezone(timezone_nw),
                                 'from': status_alias.get(item.fromString) or item.fromString,
                                 'to': status_alias.get(item.toString) or item.toString})                    
    return DataFrame(rows)

#in case we need to combine statuses
alias={
       "Ready For Development":"Ready For Development",
       "Ready for Development":"Ready For Development"
}



In [123]:

JQL = '(category in ("Shared Capabilities: CPP: Academic", "Shared Capabilities: CPP: Content Platform", "Shared Capabilities: CPP: Professional Learning", "Shared Capabilities: CPP: Research") OR filter = 48230) AND issuetype not in (subTaskIssueTypes()) AND filter = 48218 AND updatedDate >= "2021/02/09"'
transitions = read_data_sets(jql=JQL, status_alias=alias)

In [124]:
print("found",transitions.count().key,"tickets")

found 1457 tickets


In [125]:
transitions.tail()

,key,ts,from,to,epic
1452,CMH-2273,2019-10-01 03:20:32-04:00,Ready For Development,In Development,empty
1453,CMH-2273,2021-03-01 19:14:57-05:00,In Development,Closed,empty
1454,CMH-1236,2018-12-20 10:42:44-05:00,None,Backlog,NaN
1455,CMH-1236,2019-01-11 11:04:07-05:00,Open,Ready For Development,empty
1456,CMH-1236,2021-03-01 19:12:54-05:00,Ready For Development,Closed,empty


In [153]:
import pandas as pd

# Use transitions to generate the historic of a issue
def board_time(transitions):        
    last_iteration = None
    last_issue = None
    last_epic = None
    
    rows = []
    collumns = {}    
    for _, transition in transitions.sort_values(by=['key', 'ts'], ascending=True).iterrows():
        dt = pd.to_datetime(transition['ts'])              
        if transition['key'] != last_issue and last_issue is not None:
            collumns['issue'] = last_issue
            collumns['epic'] = last_epic
            rows.append(collumns)
            collumns = {}
        
        if transition['from'] is not None:
            elapsed =  dt - last_iteration            
            if transition['from'] in collumns:
                collumns[transition['from']] += elapsed 
            else:
                collumns[transition['from']] = elapsed
        else:
            collumns['started'] = dt
                
        last_iteration = dt
        last_issue = transition['key']
        last_epic = transition['epic']
    return DataFrame(rows)

df = board_time(transitions)
df.describe()

,Open,Ready For Development,In Analysis,Ready For Tech Analysis,In Tech Analysis,In Grooming,In Development,Ready For Grooming,Ready for QA,In QA,...,Ready for Code Review,In Code Review,In Dev Testing,Need Clarification,Code Review,Resolved,Reopened,On Hold,Failed QA,Failed UAT
count,68,161,173,57,12,10,138,47,54,28,...,43,11,19,11,6,15,2,30,3,1
mean,43 days 10:18:00.323529412,70 days 16:23:59.447204969,29 days 14:12:18.375722543,70 days 23:06:29.350877193,57 days 14:12:40.083333333,55 days 16:23:55.600000,27 days 03:32:03.289855072,75 days 20:24:10.425531915,13 days 14:19:54.037037037,4 days 18:43:56.392857142,...,8 days 06:54:52.023255814,4 days 05:54:55.545454545,2 days 05:01:18.105263157,88 days 13:35:21.181818182,9 days 09:37:41.833333333,16 days 20:59:51.533333333,56 days 18:58:42.500000,59 days 02:22:22.833333333,0 days 06:10:55.333333333,0 days 06:31:29
std,112 days 19:31:54.080068258,147 days 20:26:39.025611324,56 days 14:09:26.542926542,106 days 20:15:02.036649314,86 days 07:15:18.395915196,70 days 07:34:30.342867371,87 days 09:54:35.757627607,110 days 18:27:14.406004930,25 days 11:59:57.768803173,8 days 02:52:17.460936854,...,27 days 14:20:55.624111772,6 days 12:29:34.779393143,3 days 13:16:59.706083772,191 days 19:18:59.835674246,22 days 21:28:48.570593672,59 days 23:03:39.411197094,74 days 15:23:43.855135326,64 days 23:59:31.189995136,0 days 08:55:59.605133355,NaT
min,0 days 00:00:07,0 days 00:00:02,0 days 00:00:10,0 days 00:00:04,0 days 00:00:03,0 days 00:00:04,0 days 00:00:07,0 days 00:00:04,0 days 00:00:11,0 days 00:00:05,...,0 days 00:00:02,0 days 00:00:09,0 days 00:00:05,0 days 03:26:23,0 days 00:00:05,0 days 00:00:05,4 days 00:16:14,0 days 00:11:08,0 days 01:00:37,0 days 06:31:29
25%,0 days 13:35:27.500000,3 days 23:29:21,0 days 00:54:18,7 days 21:34:34,1 days 17:56:27.750000,10 days 14:26:45.750000,0 days 20:01:20.500000,9 days 01:39:40,0 days 22:47:44,0 days 00:37:46.500000,...,0 days 04:04:40,0 days 00:31:48,0 days 03:18:45.500000,4 days 07:41:55.500000,0 days 00:00:23,0 days 00:00:05.500000,30 days 09:37:28.250000,10 days 22:45:13.500000,0 days 01:01:28,0 days 06:31:29
50%,3 days 23:50:36.500000,19 days 22:11:06,5 days 20:49:42,28 days 21:48:55,7 days 12:23:15,31 days 14:25:59.500000,5 days 04:44:31,20 days 16:29:18,3 days 00:02:50.500000,0 days 17:05:23.500000,...,1 days 13:05:39,0 days 02:53:51,0 days 18:38:17,10 days 13:56:31,0 days 00:01:09.500000,0 days 00:01:30,56 days 18:58:42.500000,34 days 10:21:16.500000,0 days 01:02:19,0 days 06:31:29
75%,28 days 01:44:35,52 days 07:26:24,34 days 08:23:48,66 days 18:18:38,106 days 21:11:59.500000,78 days 19:04:19.750000,16 days 12:45:21.500000,82 days 17:08:32,14 days 08:02:46.250000,4 days 12:55:22.750000,...,5 days 23:13:03,8 days 08:38:52.500000,1 days 10:21:47,23 days 22:10:00,0 days 04:52:20.750000,0 days 00:21:45.500000,83 days 04:19:56.750000,96 days 17:52:16.250000,0 days 08:46:04.500000,0 days 06:31:29
max,599 days 02:25:57,780 days 08:08:47,314 days 16:02:09,495 days 10:14:34,236 days 19:59:04,234 days 11:25:13,584 days 18:33:52,342 days 15:25:59,142 days 01:47:28,31 days 03:44:11,...,180 days 02:45:19,15 days 03:31:13,13 days 01:28:35,615 days 17:51:45,56 days 03:14:15,232 days 19:25:50,109 days 13:41:11,212 days 11:26:49,0 days 16:29:50,0 days 06:31:29


In [143]:
df.describe().to_csv("my_description.csv")